[卷积神经网络CNN实现](https://zhuanlan.zhihu.com/p/102119808]

1. 动机
   1. 通过普通的神经网络可以实现，但是图片越来越多，如果使用NN实现则训练的参数太多
   2. 特征位置在不同的图片中会有不同的变化
2. 数据集
   1. [数据集来源](http://yann.lecun.com/exdb/mnist/)
3. 卷积

   ![avator](../resource/v2-69b4c1dd078ee363317bb8fa323eaace_b.gif)

   1. 通过卷积可以提取图片中特定线条，垂直线条或者水平线条
   ![avator](../resource/v2-ca0d5ccbaf1a30eff0f9289987486e96_720w.jpg)
   2. 填充：可以通过在周围补-实现输出前后图像大小一致
   3. 卷基层：卷基层通过一组 filter 将输入的图片转为输出的图片。卷基层的主要参数是 filter 的个数
   ![avator](../resource/v2-cad35827f01a669417b548f52dfc3c2a_720w.jpg)

In [ ]:
# 3.4 卷积层代码实现
import numpy as np

class Conv3x3:

    def __init__(self, num_filters):
        self.num_filters = num_filters
        # 初始化时除以9是因为初始值不能太大也不能太小
        self.filters = np.random.randn(num_filters, 3, 3) / 9
    

    def iterate_regions(self, image):
        h, w = image.shape